### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using aql.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Runing aql in iJava
We can run aql commands in a file. No output will be displayed. 
In _trucate_test.aql_, we have a single AQL command: TRUNCATE test



In [2]:
%sh aql -c "truncate test" -h "3.235.121.162"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.0.0</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.Operation;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpReadFlags;
import com.aerospike.client.exp.Expression;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.exp.ListExp;

AerospikeClient client = new AerospikeClient("3.235.121.162", 3000);
Key key = new Key("test", "testset", "key1");
WritePolicy wPolicy = new WritePolicy();
Bin b0 = new Bin("b00", Value.get(28)); //8byte value
Bin b1 = new Bin("b01", Value.get("8bytes018bytes028bytes038bytes048bytes058bytes068bytes078bytes08")); //64bytes value  
//Bin b3 = new Bin("b03", Value.get("8bytes018bytes028bytes038bytes048bytes058bytes068bytes078bytes08")); //64bytes value
//Bin b4 = new Bin("b04", Value.get("8bytes018bytes028bytes038bytes048bytes058bytes068bytes078bytes08")); //64bytes value
//Bin b5 = new Bin("b05", Value.get("8bytes018bytes028bytes038bytes048bytes058bytes068bytes078bytes08")); //64bytes value
//Bin b6 = new Bin("b06", Value.get("8bytes018bytes028bytes038bytes048bytes058bytes068bytes078bytes08")); //64bytes value
//Bin b7 = new Bin("b07", Value.get("8bytes018bytes028bytes038bytes048bytes058bytes068bytes078bytes08")); //64bytes value
//Bin b8 = new Bin("b08", Value.get("8bytes018bytes028bytes038bytes048bytes058bytes068bytes078bytes08")); //64bytes value
//Bin b9 = new Bin("b09", Value.get("8bytes018bytes028bytes038bytes048bytes058bytes068bytes078bytes08")); //64bytes value
//Bin b10 = new Bin("b10", Value.get("8bytes018bytes028bytes038bytes048bytes058bytes068bytes078bytes08")); //64bytes value

client.put(wPolicy, key, b0, b1);
//client.put(wPolicy, key, b0, b1, b2, b3, b4, b5, b6, b7,b8, b9, b10);
System.out.println("Record: "+ client.get(null, key)); //check record got inserted

//Using expressions to read record metadata in PI that has size of record on device
Expression recSizeExp = Exp.build(Exp.deviceSize());
Record record = client.operate( wPolicy, key,   
          ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT) 
         );
System.out.println("Record Size in bytes: " + record.getValue("size"));

//Play with bin value (val1val2... ) 
//Record size should jump in 16 byte increments on device.
// Rec size = 35 + 4 (finite ttl) + 1+7(testset)+ 1 (<128 bins) + 1+3(b00)+ 1+3(b01)...(bn)+ 2+8(int)+5+64(str)...(n) 
//          = 135 
// Nearest 16 byte boundary: 144

Record: (gen:1),(exp:417506815),(bins:(b00:28),(b01:8bytes018bytes028bytes038bytes048bytes058bytes068bytes078bytes08))
Record Size in bytes: 144


In [5]:
try {
    while(true) {
        List<Value> values = new ArrayList<>();
        //for (int i = 0; i < 1; i++) {
        for (int i = 0; i < 500; i++) {
            values.add(Value.get("111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111"));
        }
        Expression recSizeExp = Exp.build(Exp.deviceSize());
        //Record r1 = AerospikeUtil.getClient().operate(null, 
        //  new Key("dirprodv2", "size_check", "sizecheck20"), 
        //  ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT), 
        //  ListOperation.appendItems("data", values, null));
        Record r1 = client.operate(null, key, //new Key("test", "size_check", "sizecheck20"), 
                                   ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT), 
                                   ListOperation.appendItems("data", values, null)
                                  );
        System.out.println("Record Size in bytes: text " + r1.getValue("size"));

        //Record record = AerospikeUtil.getClient().operate( null, new Key("dirprodv2", "size_check", "sizecheck20"),
        //        ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT), ListOperation.size("data", null)
        //);
        
        Record record = client.operate( null, key, //new Key("test", "size_check", "sizecheck20"), 
                                       ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT), 
                                       ListOperation.size("data", null)  //<-- gives number of elements in list
                                      );       
        
        System.out.println("Record Size in bytes: text " + record.getValue("size"));
        //System.out.println("Record length " + record.getValue("data"));
        System.out.println("Bin Array (each element = #128 bytes) Size: " + record.getValue("data"));
        // ==> Record size is going up by ~64,000 bytes + msgPack overhead in each loop execution.
        
        //Thread.sleep(5);  // Tried some sleep in the loop in my initial test with i<1 instead of i<500
        //HotKey Update - loop will update the same record faster than it can be written to device
        //As record gets bigger, (>1/2 write-block-size, there is one record per write block)
        //==> device overload error
    }
} catch (Exception e) {
    e.printStackTrace();
}

Record Size in bytes: text 144
Record Size in bytes: text 67648
Bin Array (each element = #128 bytes) Size: 500
Record Size in bytes: text 67648
Record Size in bytes: text 135152
Bin Array (each element = #128 bytes) Size: 1000
Record Size in bytes: text 135152
Record Size in bytes: text 202656
Bin Array (each element = #128 bytes) Size: 1500
Record Size in bytes: text 202656
Record Size in bytes: text 270160
Bin Array (each element = #128 bytes) Size: 2000
Record Size in bytes: text 270160
Record Size in bytes: text 337648
Bin Array (each element = #128 bytes) Size: 2500
Record Size in bytes: text 337648
Record Size in bytes: text 405152
Bin Array (each element = #128 bytes) Size: 3000
Record Size in bytes: text 405152
Record Size in bytes: text 472656
Bin Array (each element = #128 bytes) Size: 3500
Record Size in bytes: text 472656
Record Size in bytes: text 540160
Bin Array (each element = #128 bytes) Size: 4000
Record Size in bytes: text 540160
Record Size in bytes: text 607648
Bi

com.aerospike.client.AerospikeException: Error 13,1,0,30000,0,0,A1 44.200.159.121 3000: Record too big
	at com.aerospike.client.command.ReadCommand.parseResult(ReadCommand.java:187)
	at com.aerospike.client.command.SyncCommand.executeCommand(SyncCommand.java:103)
	at com.aerospike.client.command.SyncCommand.execute(SyncCommand.java:63)
	at com.aerospike.client.AerospikeClient.operate(AerospikeClient.java:1798)
	at REPL.$JShell$44.do_it$($JShell$44.java:41)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at io.github.spencerpark.ijava.execution.IJavaExecutionControl.lambda$execute$1(IJavaExecutionControl.java:95)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
